In [ ]:
#%pip install --upgrade --quiet  langchain-openai
#%pip install python-dotenv --quiet
#%pip install langchain-chroma --quiet
#%pip install -U langchain-community langgraph tavily-python langgraph-checkpoint-sqlite

In [ ]:
# import getenv
import os
import getpass
import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()
# Import Azure OpenAI
from langchain_openai import AzureChatOpenAI


In [ ]:
# Correctly set environment variables using os.environ
os.environ["AZURE_OPENAI_API_VERSION"] = "2025-01-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ai-mainhub986025242733.services.ai.azure.com"
os.environ["AZURE_OPENAI_API_KEY"] = "9gLvGnTyAYZBbsmsBZbOWFJkLwe1DBJZFYtFHG3qf4Gf9Gth8jRXJQQJ99BAACfhMk5XJ3w3AAAAACOGzKp1"
os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"] = "gpt-4o"
os.environ["TAVILY_API_KEY"] = "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"

In [ ]:
import getpass
import os

if not os.environ.get("AZURE_OPENAI_API_KEY"):
  os.environ["AZURE_OPENAI_API_KEY"] = getpass.getpass("Enter API key for Azure: ")

from langchain_openai import AzureChatOpenAI

model = AzureChatOpenAI(
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    azure_deployment=os.environ["AZURE_OPENAI_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

In [ ]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Toronto")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

In [ ]:
model_with_tools = model.bind_tools(tools)

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in Vancouver?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

In [ ]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [ ]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

## Add streaming

In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

### add memory

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()


In [ ]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
# Use the agent
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="hi I am ARTURO! and i live in Oakville Ontario")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [ ]:
for step in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather where I live?")]},
    config,
    stream_mode="values",
):
    step["messages"][-1].pretty_print()